In [87]:
import pandas as pd
import requests
from pandas.io.json import json_normalize

In [84]:
data = requests.get('http://apidev.campnear.me/GetFacilitiesNear?lat=48.38&lon=-122.46&radius=10')

In [89]:
camping_json  = json.loads(data.text)
camping_df = json_normalize(camping_json['Items'])

In [91]:
camping_df.head()

,campnear_id,distance,facilitylatitude,facilitylongitude,facilityname
0,2046,5.310677,48.400246,-122.571652,Deception Pass State Park - Hope Island
1,1985,5.694163,48.341487,-122.350352,Conway
2,2935,5.990398,48.414171,-122.580001,Skagit Island State Park
3,1866,7.572723,48.488770,-122.480225,Bay View State Park
4,2048,8.521523,48.393067,-122.644658,Deception Pass State Park - Quarry Pond


In [98]:
def get_details(campnear_id):
    req_url = 'http://apidev.campnear.me/GetFacilityDetails?campnear_id=' + str(campnear_id)
    data = requests.get(req_url)
    camping_json  = json.loads(data.text)
    camping_df = json_normalize(camping_json['Items'])
    return camping_df

In [120]:
doe_bay_camping = camping_df.campnear_id.apply(lambda x: get_details(x))

In [134]:
df_out = pd.DataFrame()
for df in doe_bay_camping:
    df_out = df_out.append(pd.DataFrame(df))

In [135]:
df_out.dtype

AttributeError: 'DataFrame' object has no attribute 'dtype'

In [175]:
df_out_json = create_geojson(df_out)

In [176]:
with open('data.geojson', 'w') as fp:
    json.dump(df_out_json, fp)

In [34]:
data[data.duplicated('campnear_id')].campnear_id

Series([], Name: campnear_id, dtype: object)

In [37]:
data.to_csv('toorcamp_edit.tsv', index=False, sep='\t')

In [78]:
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize

In [174]:
def create_geojson(df, option_cols=[]):
    json_out = "{'type': 'FeatureCollection','features': [" # begin array of records
    for index,row in df.iterrows():
        json_out = json_out + "{'type': " # begin record
        json_out = json_out + "'Feature','properties': {"  # begin properties
        json_out = json_out + "'title':'" + row.facilityname + "',"
        json_out = json_out + "'description':'" + row.site_url + "',"
        for col in option_cols :
            try:
                val = row[col]
                json_out = json_out + "'" + col + "':'" + val + "',"
            except Exception as ex:
                print("create_geojoson(): Couldnt get column:" + col)
                continue
        json_out = json_out + "'marker-size':'small'"
        json_out = json_out + '},' # end properties
        
        #geo point
            
        json_out = json_out + "'geometry': {'type': 'Point','coordinates':[" + str(row.facilitylongitude) \
        + ',' + str(row.facilitylatitude) + ']}'
    
        json_out = json_out + '},' # end record 
    
    # remove last comma
    json_out = json_out[0:len(json_out)-1]
    json_out = json_out + "]}"
    return json_out

In [66]:
def replace_nan(value):
        try :
                if np.isnan(value):
                        return None
        except:
                err = "value is not NaN convertable"

        if (value == 'NaN'):
                return None
        else:
                return value

def process_data(df_items):
        try:
                items_list=[]
                for index,row in df_items.iterrows():
                        item={}
                        for col,value in row.iteritems():
                                val = replace_nan(value)
                                item.update({col: val})
                        items_list.append(item)
                return {'StatusCode':'200','NumRecords':df_items.campnear_id.count() ,'Items':items_list}

        except Exception as ex:
                return {'error':str(ex)}


In [126]:
df_items = pd.read_sql("select * from toorcamp", engine)
df_out = process_data(df_items)
with open('data.json', 'w') as fp:
    json.dump(df_out, fp)


In [164]:
str_test = "the thing"

In [167]:
str_test[0:len(str_test)-1]

'the thin'

In [68]:
import json
with open('data.json', 'w') as fp:
    json.dump(df_out, fp)

In [72]:
with open('data.json') as data_file:    
    data = json.load(data_file)

In [76]:
data['Items']

[{u'ada_standard': None,
  u'ada_standard_full': None,
  u'ada_tent': None,
  u'amphitheater': None,
  u'barbeque': None,
  u'cabin': None,
  u'cabin_deluxe': None,
  u'cabin_electric': None,
  u'cabin_nonelectric': None,
  u'cabins_available': u'false',
  u'campnear_id': 1,
  u'cg_accessible': u'false',
  u'cg_conditions': None,
  u'cg_fcfs': u'false',
  u'cg_flush': u'false',
  u'cg_group': None,
  u'cg_lookout': None,
  u'cg_nearbeach': None,
  u'cg_nearlake': None,
  u'cg_nearriver': None,
  u'cg_petsallowed': u'false',
  u'cg_proximitytowater': None,
  u'cg_reservable': u'true',
  u'cg_season': u'5/1 - 9/28/2015',
  u'cg_shower': u'false',
  u'cg_solitude': u'false',
  u'cg_status': None,
  u'cg_transit_avail': u'false',
  u'cg_usage': u'medium',
  u'cg_vault': u'true',
  u'cg_water': u'false',
  u'drivehours': u'2.0',
  u'drivemiles': u'123.0',
  u'driveminutes': u'16.0',
  u'elevation': u" 1768'",
  u'end_date': u'06/11/2016',
  u'equestrian_nonelectric': None,
  u'facilityid': 

In [80]:
camping_df = json_normalize(data['Items'])

In [82]:
camping_df.head()

,ada_standard,ada_standard_full,ada_tent,amphitheater,barbeque,cabin,cabin_deluxe,cabin_electric,cabin_nonelectric,cabins_available,...,tent_site,total_sites,totem_cabin,transithours,transitminutes,walk_to,youth_camp,yurt,yurt_deluxe,yurts_available
0,None,None,None,None,None,None,None,None,None,false,...,None,16,None,None,None,None,None,None,None,false
1,None,None,None,None,None,None,None,None,None,false,...,None,32,None,None,None,None,None,None,None,false
2,None,None,None,None,None,None,None,None,None,false,...,None,13,None,None,None,None,None,None,None,false
3,None,None,None,None,None,None,None,None,None,false,...,None,None,None,None,None,None,None,None,None,false
4,None,None,None,None,None,None,None,None,None,false,...,None,9,None,None,None,None,None,None,None,false
